In [1]:
import healpy
import Tools




The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:

#basedir, tag, galdefdir, = sys.argv[1:4]
#fname = basedir+'/'+tag+'_XCO.hdf5'
import Analysis
# Load the analysis
A = Analysis.Analysis(tag='P7REP_CLEAN_V15_calore', fglpath='/data/gll_psc_v14.fit',  
templateDir='/data/Extended_archive_v15/Templates', basepath='/data/GCE_sys/')
A.BinPhotons(infile='binned_photons_all_sky.npy')
# Load 2FGL 
A.AddPointSourceTemplate(fixNorm=True, pscmap=('PSC_all_sky_3fgl.npy'))
A.CalculatePixelWeights(diffuse_model='fermi_diffuse_'+A.tag+'.npy',psc_model='PSC_' + A.tag + '.npy',
                        alpha_psc=5., f_psc=0.05)
A.AddIsotropicTemplate(fixNorm=False, fixSpectrum=True) # External chi^2 used to fix normalization within uncertainties
A.PopulateROI([0,0],radius=360, fix_radius=360., include_point=False)

A.AddFermiBubbleTemplate(template_file='./bubble_templates_diskcut30.0.fits', 
                     spec_file='./reduced_bubble_spec_apj_793_64.dat', fixSpectrum=True, fixNorm=True)





Populating ROI with point sources: 123 of 3034
Populating ROI with point sources: 603 of 3034
Populating ROI with point sources: 640 of 3034
Populating ROI with point sources: 697 of 3034
Populating ROI with point sources: 930 of 3034
Populating ROI with point sources: 956 of 3034
Populating ROI with point sources: 1001 of 3034
Populating ROI with point sources: 1568 of 3034
Populating ROI with point sources: 1624 of 3034
Populating ROI with point sources: 1848 of 3034
Populating ROI with point sources: 1963 of 3034
Populating ROI with point sources: 1966 of 3034
Populating ROI with point sources: 2020 of 3034
Populating ROI with point sources: 2120 of 3034
Populating ROI with point sources: 2275 of 3034
Populating ROI with point sources: 2287 of 3034
Populating ROI with point sources: 2355 of 3034
Populating ROI with point sources: 2393 of 3034
Populating ROI with point sources: 2406 of 3034
Populating ROI with point sources: 2441 of 3034
Populating ROI with point sources: 2500 of 303

In [5]:
A.PrintTemplates()

                NAME                    LIMITS      VALUE    FIXNORM    FIXSPEC   SRCCLASS
             PuppisA              [None, None]  1.000e+00       True       True        FGL
       HESSJ1614n518              [None, None]  1.000e+00       True       True        FGL
       HESSJ1616n508              [None, None]  1.000e+00       True       True        FGL
           CenALobes              [None, None]  1.000e+00       True       True        FGL
       HESSJ1632n478              [None, None]  1.000e+00       True       True        FGL
       HESSJ1837n069              [None, None]  1.000e+00       True       True        FGL
        FermiDiffuse              [None, None]  1.000e+00      False       True        GEN
                 SMC              [None, None]  1.000e+00       True       True        FGL
                 W28              [None, None]  1.000e+00       True       True        FGL
                 W44              [None, None]  1.000e+00       True       True        FGL

In [9]:


import Tools

def FitDiffuseModel(diffuse_model):
    hpix = [Tools.SampleCartesianMap(diffuse_model, E_min=A.bin_edges[i], E_max=A.bin_edges[i+1], nside=256, E_bins=3) for i,E in enumerate(A.central_energies)]

    for key, t in A.templateList.items():
        t.fixSpectrum = True
        if t.sourceClass == 'FGL':
            t.fixNorm=True

    A.AddTemplate('FermiDiffuse', np.array(hpix), fixSpectrum=True, )


    fval, res = [], []

    print 'Running Local Ring Fit...'
    A.GenSquareMask(l_range=[-180.,180], b_range=[-80.,80.], plane_mask=8.)
    m, R = A.RunLikelihood( print_level=1, precision=None, tol=1e3, force_cpu=True)[:2]
    fval.append(m.fval) 
    res.append(R)

    A.templateList['Isotropic'].fixNorm = True
    print 'Running Inner Rings Fit...'
    A.GenSquareMask(l_range=[-80,80], b_range=[-8.,8.], plane_mask=0)
    m, R = A.RunLikelihood( print_level=1, precision=None, tol=1e3)[:2]
    fval.append(m.fval)
    res.append(R)


    print 'Running Outer Rings Fit...'
    A.GenSquareMask(l_range=[-180,-80], b_range=[-8.,8.], plane_mask=0)
    A.GenSquareMask(l_range=[80,180], b_range=[-8.,8.], plane_mask=0, merge=True)

    m, R = A.RunLikelihood( print_level=1, precision=None, tol=1e3)[:2]
    fval.append(m.fval)
    res.append(R)
    
    return fval, R


results = {}

results['P6v11'] = FitDiffuseModel('/data/fermi_background_models/gll_iem_v02_P6_V11_DIFFUSE.fit')
results['P7v6'] = FitDiffuseModel('/data/fermi_background_models/gal_2yearp7v6_v0.fits')
results['P7REP'] = FitDiffuseModel('/data/fermi_background_models/gll_iem_v05_rev1.fit')




Running Local Ring Fit...
Masking completed in 1.8444e+01 s
Code generation completed in 1.5114e-02 s
Using GPU mode. (Successful import of cudamat module.)


FCN = 4019504.23865 
 TOTAL NCALL = 20 
 NCALLS = 20 
 
 
 EDM = 1.07329328044e-05 
 GOAL EDM = 0.05 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,PuppisA,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
2,HESSJ1614n518,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
3,HESSJ1616n508,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
4,CenALobes,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
5,HESSJ1632n478,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
6,HESSJ1837n069,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
7,FermiDiffuse,8.910065e-01,3.839542e-04,0.000000e+00,0.000000e+00,,,
8,SMC,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
9,W28,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
10,W44,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED


Migrad completed fitting   8.06e+01 s
Running Inner Rings Fit...
Masking completed in 6.9487e+00 s
Code generation completed in 1.5105e-02 s
Using GPU mode. (Successful import of cudamat module.)


FCN = -8681739.87739 
 TOTAL NCALL = 36 
 NCALLS = 32 
 
 
 EDM = 1.14170620844 
 GOAL EDM = 0.05 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 False 
 True 
 False 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 True 
 
 False

...

FCN = -4971541.87739 
 TOTAL NCALL = 35 
 NCALLS = 28 
 
 
 EDM = 7.8125 
 GOAL EDM = 0.05 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 False 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 True 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,PuppisA,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
2,HESSJ1614n518,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
3,HESSJ1616n508,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
4,CenALobes,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
5,HESSJ1632n478,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
6,HESSJ1837n069,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
7,FermiDiffuse,8.676715e-01,4.328133e-04,0.000000e+00,0.000000e+00,,,
8,SMC,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
9,W28,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED
10,W44,1.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,,,FIXED


ERROR: AttributeError: 'module' object has no attribute 'save' [IPython.core.interactiveshell]
ERROR:astropy:AttributeError: 'module' object has no attribute 'save'


Migrad completed fitting   2.89e+00 s


AttributeError: 'module' object has no attribute 'save'

In [11]:
import cPickle as pickle
pickle.dump(results, open('fermi_background_fits.pickle','wb'))


In [12]:

print results


{'P6v11': ([4019504.2386466544, -8681739.87739326, -4965540.37739326], None), 'P7REP': ([4035750.811484296, -8685103.87739326, -4971541.87739326], None), 'P7v6': ([4032585.6077633617, -8678603.87739326, -4966474.37739326], None)}
